<a href="https://colab.research.google.com/github/sheon-j/nlp-study/blob/main/tensorflow-nlp-tutorial/week05_recurrent_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Study: Week 5 - Recurrent Neural Network

[딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/book/2155) 스터디

---

**Contents**
1. 순환 신경망(Recurrent Neural Network)
2. 장단기 메모리(Long Short-Term Memory, LSTM)

## 1. 순환 신경망(Recurrent Neural Network)

* 피드 포워드 신경망의 한계
  * 피드 포워드 신경망은 **입력의 길이가 고정**되어 자연어 처리에 한계
  * 다양한 길이의 입력 시퀀스를 처리할 수 있는 인공 신경망이 필요
* 순환 신경망(Recurrent Neural Network, RNN) 
  * 자연어 처리에 대표적으로 사용되는 인공 신경망
    * 재귀 신경망(Recursive Neural Network)은 전혀 다른 개념
  * 입력과 출력을 시퀀스 단위로 처리하는 **시퀀스(Sequence) 모델**
    * 입력 시퀀스 => 출력 시퀀스 (번역기)
  * RNN 모델
    * 기본적인 순환 신경망인 바닐라 RNN
    * 바닐라 RNN의 단점을 개선한 LSTM, GRU

### 1.1. 순환 신경망(Recurrent Neural Network)

* RNN과 FFNN의 비교
  * FFNN: 은닉층에서 활성화 함수의 결과를 출력층으로 보냄

    ![img](https://wikidocs.net/images/page/24987/mlp_final.PNG)
    
  * RNN: 은닉층에서 활성화 함수의 결과를 **출력층** 방향으로도, 다시 은닉층 노드의 다음 계산의 **입력**으로도 보냄

    ![img](https://wikidocs.net/images/page/24987/rnn_final.PNG)

* RNN의 구성
    
    ![img](https://wikidocs.net/images/page/22886/rnn_image3.5.PNG)
    
    ![img](https://wikidocs.net/images/page/22886/rnn_image1_ver2.PNG)

  * 입력 벡터 $x$
  * 출력 벡터 $y$
  * 셀 (Cell)
    * 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드
    * 이전의 값을 기억하려고 하는 일종의 메모리 역할도 수행(RNN셀/메모리셀)

* 메모리 셀

  ![img](https://wikidocs.net/images/page/22886/rnn_image2_ver3.PNG)
  * 메모리 셀은 바로 이전 시점 은닉층의 메모리 셀에서 나온 값을 입력으로 사용(재귀적 활동)
  * 현재 시점을 변수 $t$로 표현
    * $t$에서의 메모리 셀의 값은 과거의 메모리 셀들의 값에 영향을 받은 것을 의미
  * **은닉 상태**: 메모리 셀이 출력층 방향 또는 다음 시점인 t+1의 자신에게 보내는 값
    * $t$ 시점의 메모리 셀은 $t-1$ 시점의 메모리 셀이 보낸 은닉 상태값을 은닉 상태 계산을 위한 입력값으로 사용

* RNN의 뉴런 단위 시각화

  ![img](https://wikidocs.net/images/page/22886/rnn_image2.5.PNG)

  * RNN 시점: 입력 벡터 차원 4, 은닉 상태 2, 출력 백터 차원 2
  * 뉴런 단위: 입력층 뉴런 수 4, 은닉층 뉴런 수 2, 출력층 뉴런 수 2

* 입출력에 따른 RNN 분류

  ![img](https://wikidocs.net/images/page/22886/rnn_image3_ver2.PNG)

  * 입출력의 단위는 **단어 벡터**
  * 일 대 다 구조: 하나의 시퀀스 입력에 대해 여러 개의 시퀀스 출력 (이미지 캡셔닝, [퀵 드로우](https://quickdraw.withgoogle.com/?locale=ko))
  * 다 대 일 구조: 여러 단어 시퀀스에 대해 하나의 출력 (긍정/부정 분류, 스팸메일 분류 등)
  * 다 대 다 구조: 문장을 입력하면 문장을 출력 (챗봇, 번역기, 태깅)

* RNN 수식

  ![img](https://wikidocs.net/images/page/22886/rnn_image4_ver2.PNG)

  * 현재 시점: $t$
  * 은닉 상태 값: $h_t$
  * 은닉층: $h_t = tanh(W_xx_t + W_hh_{t-1}+b)$
  * 출력층: $y_t = f(W_yh_t + b)$<br>단, $f$는 비선형 활성화 함수 중 하나

* 출력층의 활성화 함수
  * 비선형 함수를 사용하는 이유
    * 선형 함수인 $ℎ(𝑥)=𝑐𝑥$를 활성 함수로 사용한 3층 네트워크를 가정
    * 이를 식으로 나타내면 $𝑦(𝑥)=ℎ(ℎ(ℎ(𝑥)))$
      * 이 계산은 $𝑦(𝑥)=𝑐∗𝑐∗𝑐∗𝑥$처럼 세번의 곱셈을 수행
      * $𝑎=𝑐3$ 또는 $𝑦(𝑥)=𝑎𝑥$와 똑같음
      * **즉 은닉층이 없는 네트워크**
    * 층을 쌓는 혜택을 얻으려면 활성함수로는 반드시 비선형함수를 사용해야 함
  * 분류에 따른 활성화 함수
    * 이진 분류: 출력층에 로지스틱 회귀를 사용하여 시그모이드 함수를 사용
    * 다중 클래스 분류: 출력층에 소프트맥스 회귀를 사용하여로 소프트맥스 함수를 사용할 수 있습니다.






### 1.2. 케라스(Keras)로 RNN 구현하기

* RNN층 추가하는 코드
```python
from tensorflow.keras.layer import SimpleRNN
...
# 기본 구성
moddel.add(SimpleRNN(hidden_units))
# 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_units, input_shape=(timesteps, input_dim)))
# 다른 표기
model.add(SimpleRNN(hidden_units, input_length=M, input_dim=N))
```
​    ![img](https://wikidocs.net/images/page/22886/rnn_image6between7.PNG)
* RNN의 파라미터    
  * **hidden_units (output_dim)**
    * 은닉 상태의 크기: RNN의 용량(capacity)을 늘림
    * t의 메모리 셀이 t+1 메모리 셀 혹은 출력층으로 보내는 값의 크기
    * 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가짐
  * **input_dim**: 입력의 크기
  * **timesteps (input_length)**: 입력 시퀀스의 길이. 시점의 수
  * **batch_size**: 한 번에 학습하는 데이터의 개수
  * RNN 층(hidden_units)은 3D 텐서(input_dim X timesteps X batch_size)를 입력 받음
    
* RNN의 출력 방식

  ![img](https://wikidocs.net/images/page/22886/rnn_image8_ver2.PNG)

  * RNN 층은 사용자의 설정에 따라 두 가지 종류의 출력 방식
    * RNN 층의 return_sequences (True/Fasle) 파라미터로 설정이 가능
    * output_dim은 앞서 코드에서 정의한 hidden_units의 값으로 설정됨
  * return_sequence=False: 최종 시점 메모리셀이 (batch_size, output_dim) 크기의 2D 은닉 상태 값 출력 (다 대 일)
  * return_sequence=True: 모든 시점의 메모리셀들이 (batch_size, timesteps, output_dim) 크기의 3D 은닉 상태 값 출력 (다 대 다)

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
# batch_size=None => Output Shape이 (None, 3)
model.add(SimpleRNN(3, input_length=2, input_dim=10))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [3]:
model = Sequential()
# batch_size=10 => Output Shape이 (8, 3)
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [4]:
model = Sequential()
# 다 대 다 출력
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (8, 2, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


### 1.3. 파이썬으로 RNN 구현하기
* 은닉 상태 값 출력 실습
$$h_t=tanh(W_xX_t+W_hh_{t-1}+b)$$

In [5]:
'''
RNN 모델 값 설정
'''
import numpy as np

timesteps = 10
input_dim = 4
hidden_units = 8

# 입력에 해당되는 2D 텐서
# 실제는 (batch_size, timesteps, input_dim)의 크기의 3D 텐서를 입력
inputs = np.random.random((timesteps, input_dim))

# 초기 은닉 상태는 0(벡터)로 초기화
hidden_state_t = np.zeros((hidden_units,)) 

print('초기 은닉 상태 :',hidden_state_t)

초기 은닉 상태 : [0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
'''
가중치, 편향 초기화
'''
Wx = np.random.random((hidden_units, input_dim))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_units, hidden_units)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_units,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

print('가중치 Wx의 크기(shape) :',np.shape(Wx))
print('가중치 Wh의 크기(shape) :',np.shape(Wh))
print('편향의 크기(shape) :',np.shape(b))

가중치 Wx의 크기(shape) : (8, 4)
가중치 Wh의 크기(shape) : (8, 8)
편향의 크기(shape) : (8,)


In [7]:
total_hidden_states = []

# 각 시점 별 입력값
for input_t in inputs:

  # Wx * Xt + Wh * Ht-1 + b(bias)
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b)

  # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep t, output_dim)
  # 각 시점의 은닉 상태의 값을 계속해서 누적
  total_hidden_states.append(list(output_t))
  hidden_state_t = output_t

# 출력 시 값을 깔끔하게 해주는 용도.
total_hidden_states = np.stack(total_hidden_states, axis = 0) 

# (timesteps, output_dim)
print('모든 시점의 은닉 상태 :')
print(*total_hidden_states, sep='\n')

모든 시점의 은닉 상태 :
[0.90971495 0.77944061 0.88988148 0.87215405 0.91354179 0.90013444
 0.8592608  0.89952046]
[0.99999759 0.99999098 0.99909432 0.9998674  0.99990457 0.99920983
 0.99959345 0.99986888]
[0.99999977 0.9999986  0.9997985  0.99995528 0.99997987 0.99966424
 0.9999496  0.99997004]
[0.9999999  0.99999904 0.99993008 0.99998157 0.99999122 0.99986312
 0.99998856 0.99998739]
[0.99999989 0.999999   0.99989219 0.99996683 0.99998418 0.99983907
 0.99998407 0.9999836 ]
[0.99999964 0.99999878 0.99984716 0.99997242 0.99998103 0.99975987
 0.99996681 0.99998092]
[0.99999966 0.99999871 0.99981175 0.99995913 0.9999779  0.99967328
 0.99995763 0.99997541]
[0.99999944 0.99999822 0.99960813 0.99993443 0.9999659  0.99941626
 0.99986483 0.9999523 ]
[0.99999996 0.99999877 0.9999042  0.99998231 0.99999126 0.99990701
 0.9999792  0.99998079]
[0.99999969 0.99999851 0.99981384 0.99997959 0.9999823  0.99980987
 0.99994442 0.99997661]


### 1.4. 깊은 순환 신경망(Deep Recurrent Neural Network)

![img](https://wikidocs.net/images/page/22886/rnn_image4.5_finalPNG.PNG)

* 다수의 은닉층을 가진 RNN
* 위 그림은 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 순환 신경망

* 코드 구현

In [8]:
# 은닉층을 2개 추가한 코드
model = Sequential()
# 첫번째 은닉층의 은닉 상태 값을 다음 은닉층에 보내야하므로 return_sequences = True를 설정
model.add(SimpleRNN(hidden_units, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_units, return_sequences=True))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 10, 8)             112       
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 10, 8)             136       
                                                                 
Total params: 248
Trainable params: 248
Non-trainable params: 0
_________________________________________________________________


### 1.5. 양방향 순환 신경망(Bidirectional Recurrent Neural Network)
* 양방향 순환 신경망 아이디어

  * 시점 t에서의 출력값을 예측할 때 이전 시점의 입력뿐만 아니라, 이후 시점의 입력 또한 예측에 기여

  * 양방향 RNN은 이전과 이후의 시점 모두를 고려해서 현재 시점의 예측을 더욱 정확하게 함

    ```
    운동을 열심히 하는 것은 [        ]을 늘리는데 효과적이다.
    
    1) 근육
    2) 지방
    3) 스트레스
    ```
    ![img](https://wikidocs.net/images/page/22886/rnn_image3.5.PNG)
    ![img](https://wikidocs.net/images/page/22886/rnn_image5_ver2.PNG)

* 양방향 RNN 구조

  * 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용
  * 첫번째 메모리 셀은 ***t-1*의 은닉 상태(Forward States)** 를 전달받아 *t*의 은닉 상태 계산
  * 두번째 메모리 셀은 ***t+1*의 은닉 상태(Forward States)** 를 전달받아 *t*의 은닉 상태 계산
  * 이 두 개의 값 모두 현재 시점의 출력층에서 출력값을 예측하기 위해 사용

In [9]:
from tensorflow.keras.layers import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 10, 16)           224       
 l)                                                              
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________


* 깊은 양방향 RNN

  ![img](https://wikidocs.net/images/page/22886/rnn_image6_ver3.PNG)

  * 양방향 RNN도 다수의 은닉층을 가질 수 있음
    * 위 그림은 은닉층이 2개인 깊은(deep) 양방향 순환 신경망
    * 양방향 RNN은 태깅 작업 챕터에서 실습
  * 은닉층이 무조건 많다고 해서 모델의 성능이 좋아지는 것은 아님
    * 은닉층을 추가하면 학습할 수 있는 양은 많아지지만, 훈련 데이터 또한 많은 양이 필요

  * 아래의 코드는 은닉층이 4개인 RNN



In [10]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 10, 16)           224       
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
Total params: 1,424
Trainable params: 1,424
Non-traina

### 1.6. 점검 퀴즈

* 모델에 대한 설명이 다음과 같을 때, 총 파라미터 개수를 구하여라.

  1. Embedding을 사용하며, 단어 집합(Vocabulary)의 크기가 5,000이고 임베딩 벡터의 차원은 100
  2. 은닉층에서는 Simple RNN을 사용하며, 은닉 상태의 크기는 128
  3. 훈련에 사용하는 모든 샘플의 길이는 30으로 가정
  4. 이진 분류를 수행하는 모델로, 출력층의 뉴런은 1개로 시그모이드 함수를 사용
  5. 은닉층은 1개

* 참고

  ![img](https://wikidocs.net/images/page/22886/rnn_images4-5.PNG)

In [11]:
from keras.layers import Embedding, Dense

vocab_size = 5000
embedding_dim = 100
hidden_size = 128

model = Sequential()
# 입력 벡터
model.add(Embedding(vocab_size, embedding_dim)) # 100*5,000
# 메모리 셀 (ℎ𝑡 = 𝑊𝑋+𝑊ℎ+𝑏)
model.add(SimpleRNN(hidden_size))               # (128*100)+(128*128)+128
# 출력 벡터
model.add(Dense(1, activation='sigmoid'))       # 128+1
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         500000    
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 128)               29312     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 529,441
Trainable params: 529,441
Non-trainable params: 0
_________________________________________________________________


---
## 2. 장단기 메모리(Long Short-Term Memory, LSTM)

* 가장 단순한 형태의 RNN을 바닐라 RNN(Vanilla RNN)(케라스에서는 SimpleRNN)
* 바닐라 RNN의 한계를 극복하기 위해 LSTM과 같은 다양한 RNN의 변형이 나옴 

### 2.1. 바닐라 RNN의 한계

* 출력 결과가 이전의 계산 결과에 의존

  ![img](https://wikidocs.net/images/page/22888/lstm_image1_ver2.PNG)
  * 바닐라 RNN은 비교적 짧은 시퀀스(sequence)에 대해서만 효과
  * 바닐라 RNN의 시점(time step)이 길어질 수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생
  * 뒤로 갈수록 x1의 정보량은 손실되고, 시점이 충분히 긴 상황에서는 x1의 전체 정보에 대한 영향력은 거의 의미가 없을 수도 있음
* 문서의 특성(두괄식)에 따라 가장 중요한 정보가 시점의 앞 쪽에 위치할 수도 있음
  ```
  모스크바에 여행을 왔는데 건물도 예쁘고 먹을 것도 맛있었어. 
  그런데 글쎄 직장 상사한테 전화가 왔어. 
  어디냐고 묻더라구 그래서 나는 말했지. 
  저 여행왔는데요. 여기 ______
  ```
  * 다음 단어를 예측하기 위해서는 장소 정보가 필요. 장소 정보에 해당되는 단어인 '모스크바'는 앞에 위치
  * RNN이 충분한 기억력을 가지고 있지 못한다면 다음 단어를 엉뚱하게 예측

* 이를 **장기 의존성 문제(the problem of Long-Term Dependencies)**라 함

### 2.2. 바닐라 RNN 내부 열어보기

![img](https://wikidocs.net/images/page/22888/vanilla_rnn_ver2.PNG)

* RNN 계열의 인공 신경망의 그림에서는 편향 b를 생략합니다
  * 편향 b를 그린다면 xt 옆에 tanh로 향하는 또 하나의 입력선

* $h_t=tanh(W_xx_t+W_hh_{t−1}+b)$
  * $x_t$와 $h_{t−1}$이라는 두 개의 입력이 각 가중치$w$와 곱해져서 메모리 셀의 입력이 됨
  * 이를 하이퍼볼릭탄젠트 활성 함수의 결과로 은닉층의 출력인 은닉 상태가 됨

### 2.3. LSTM(Long Short-Term Memory)

* LSTM의 내부

  ![img](https://wikidocs.net/images/page/22888/vaniila_rnn_and_different_lstm_ver2.PNG)
  * 장단기 메모리(Long Short-Term Memory, LSTM): 바닐라 RNN의 장기 의존성 문제를 보완한 RNN의 일종
    * RNN과 비교하여 긴 시퀀스의 입력을 처리하는데 탁월한 성능
  * LSTM은 은닉층의 메모리 셀: 불필요한 기억을 지우고, 기억해야할 것들을 정함
    * 입력 게이트
    * 삭제 게이트
    * 출력 게이트
  * 셀 상태(cell state): $t$시점의 셀 상태를 $C_t$로 표현

* 셀 상태

  ![img](https://wikidocs.net/images/page/22888/cellstate.PNG)

  * 은닉 상태처럼 이전 시점의 셀 상태가 다음 시점의 셀 상태를 구하기 위한 입력으로서 사용
  * 은닉 상태의 값과 셀 상태의 값을 구하기 위해서 새로 추가 된 3개의 게이트를 사용
    * 각 게이트는 공통적으로 시그모이드 함수가 존재
    * 시그모이드 함수를 지난 0과 1사이의 출력 값으로 게이트를 조절
  * 셀 상태의 수식
    - $σ$: 시그모이드 함수
    - $tanh$: 하이퍼볼릭탄젠트 함수
    - $W_{xi},W_{xg},W_{xf},W_{xo}$: $x_t$와 함께 각 게이트에서 사용되는 4개의 가중치
    - $W_{hi},W_{hg},W_{hf},W_{ho}$: $h_{t−1}$와 각 게이트에서 사용되는 4개의 가중치
    - $b_i,b_g,b_f,b_o$: 각 게이트에서 사용되는 4개의 편향

#### **(1) 입력 게이트**

![img](https://wikidocs.net/images/page/22888/inputgate.PNG)

* 현재 정보를 기억하는 게이트
* **$i_t=σ(W_{xi}x_t+W_{hi}h_{t−1}+b_i)$**
  * $t$(현 시점)의 $x$값과 입력 게이트로 이어지는 가중치 $Wxi$를 곱한 값 
  * $t-1$(이전 시점)의 은닉 상태가 입력 게이트로 이어지는 가중치 $W_{hi}$를 곱한 값
  * 이 둘을 더하여 시그모이드 함수를 지남 (0~1 사이의 값)
* **$g_t=tanh(W_{xg}x_t+W_{hg}h_{t−1}+b_g)$**
  * $t$의 $x$값과 입력 게이트로 이어지는 가중치 $Wxg$를 곱한 값 
  * $t-1$의 은닉 상태가 입력 게이트로 이어지는 가중치 $W_{hg}$를 곱한 값
  * 이 둘을 더하여 하이퍼볼릭탄젠트 함수를 지남 (-1~1 사이의 값)
* 이 두 개의 값을 가지고 이번에 선택된 기억할 정보의 양을 정함
  * 구체적으로 어떻게 결정하는지는 아래에서 배우게 될 셀 상태 수식을 참고

#### **(2) 삭제 게이트**

![img](https://wikidocs.net/images/page/22888/forgetgate.PNG)

* 기억을 삭제하기 위한 게이트
* **$f_t=σ(W_{xf}x_t+W_{hf}h_{t−1}+b_f)$**
  * $t$의 $x$값과 $t-1$의 은닉 상태가 시그모이드 함수를 지남 (0~1 사이의 값)
  * 이 값이 삭제 과정을 거친 정보의 양 (0 정보 삭제, 1 정보 기억)
* 이를 가지고 셀 상태를 구함
  * 구체적으로 어떻게 결정하는지는 아래에서 배우게 될 셀 상태 수식을 참고

#### **(3) 셀 상태**

![img](https://wikidocs.net/images/page/22888/cellstate2.PNG)

* 삭제 게이트에서 일부 기억을 잃은 상태
* **$C_t=f_t∘C_{t−1}+i_t∘g_t$**
  * 입력 게이트에서 구한 $i_t, g_t$ 의 값에 원소별 곱(entrywise product). 선택된 기억할 값이라 함
    * 같은 크기의 두 행렬이 있을 때 같은 위치의 성분끼리 곱
    *  ∘ 로 표현
  * 선택된 기억할 값에 삭제 게이트의 결과값과 더함. t의 셀 상태라고 함
    * 삭제 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미
    * 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정
* 이 값은 다음 t+1의 LSTM 셀로 넘겨짐

#### **(4) 출력 게이트와 은닉 상태**

![img](https://wikidocs.net/images/page/22888/outputgateandhiddenstate.PNG)

* 출력 게이트
  * **$o_t=σ(W_{xo}x_t+W_{ho}h_{t−1}+b_o)$**
    * t의 x값과 t-1의 은닉 상태가 시그모이드 함수를 지난 값
  * 해당 값은 현재 시점 t의 은닉 상태를 결정

* 은닉 상태
  * **$h_t=o_t∘tanh(c_t)$**
    * 셀 상태의 값이 하이퍼볼릭탄젠트 함수를 지남 (-1~1사이의 값)
    * 이 값은 출력 게이트의 값과 연산. 값이 걸러지는 효과가 발생하여 은닉 상태가 됨
  * 은닉 상태의 값은 또한 출력층으로도 향함